In [4]:
import pandas as pd
from pathlib import Path

def process_in_batches(base_dir, output_dir, batch_size=12000):
    """
    Process data in memory-efficient batches with specified output directory
    """
    base_path = Path(base_dir)
    output_path = Path(output_dir)
    
    # Create output directory if it doesn't exist
    output_path.mkdir(exist_ok=True)
    
    print(f"Starting batch processing...")
    print(f"Output files will be saved in: {output_path}")
    
    # Read main CSV in batches
    csv_path = base_path / "usecase_3_.csv"
    
    # Initialize an empty list to store processed chunks
    processed_chunks = []
    chunk_count = 0
    
    def safe_join(x):
        return '; '.join(sorted(set(str(v) for v in x if pd.notna(v) and str(v).strip())))
    
    for chunk in pd.read_csv(csv_path, chunksize=batch_size):
        chunk_count += 1
        print(f"Processing chunk {chunk_count}...")
        
        # Rename the NCT Number column
        chunk = chunk.rename(columns={"NCT Number": "nct_id"})
        
        # Process each text file
        for txt_file in ["drop_withdrawals.txt", "eligibilities.txt", 
                        "facilities.txt", "reported_events.txt"]:
            print(f"  Merging with {txt_file}...")
            
            # Read text file
            txt_path = base_path / txt_file
            df_txt = pd.read_csv(txt_path, sep='|', dtype=str)
            
            # Get list of columns to aggregate (excluding nct_id)
            agg_columns = [col for col in df_txt.columns if col != 'nct_id']
            
            # Create aggregation dictionary
            agg_dict = {col: safe_join for col in agg_columns}
            
            # Aggregate text file data to prevent cartesian product
            df_txt_agg = df_txt.groupby('nct_id').agg(agg_dict).reset_index()
            
            # Add suffix to prevent column name conflicts
            suffix = f"_{txt_file.split('.')[0]}"
            df_txt_agg.columns = [col if col == 'nct_id' else f"{col}{suffix}" 
                                for col in df_txt_agg.columns]
            
            # Merge with current chunk
            chunk = chunk.merge(df_txt_agg, on='nct_id', how='left')
            
            # Clear memory
            del df_txt
            del df_txt_agg
        
        processed_chunks.append(chunk)
        
        # Save intermediate results every 5 chunks
        if chunk_count % 5 == 0:
            print(f"Saving intermediate results for chunks 1-{chunk_count}...")
            interim_df = pd.concat(processed_chunks, ignore_index=True)
            interim_path = output_path / f"merged_data_interim_{chunk_count}.csv"
            interim_df.to_csv(interim_path, index=False)
            processed_chunks = [interim_df]  # Keep only the concatenated result
        
        # Clear memory
        del chunk
    
    print("Concatenating all processed chunks...")
    final_df = pd.concat(processed_chunks, ignore_index=True)
    
    # Save final result
    final_output_path = output_path / "merged_data_final.csv"
    print(f"Saving final merged data to {final_output_path}")
    final_df.to_csv(final_output_path, index=False)
    
    print(f"Process completed! Final file saved at: {final_output_path}")
    print(f"Final dataset dimensions: {final_df.shape}")
    return final_output_path

if __name__ == "__main__":
    # Set your base directory and output directory
    base_dir = r"C:\Users\Siddhant Nijhawan\Downloads\NEST\Problem Statements and Data Sets"
    output_dir = r"C:\Users\Siddhant Nijhawan\Downloads\NEST\Problem Statements and Data Sets\new csv"
    
    try:
        output_path = process_in_batches(base_dir, output_dir)
    except Exception as e:
        print(f"Error occurred: {str(e)}")

Starting batch processing...
Output files will be saved in: C:\Users\Siddhant Nijhawan\Downloads\NEST\Problem Statements and Data Sets\new csv
Processing chunk 1...
  Merging with drop_withdrawals.txt...
  Merging with eligibilities.txt...
  Merging with facilities.txt...
  Merging with reported_events.txt...
Processing chunk 2...
  Merging with drop_withdrawals.txt...
  Merging with eligibilities.txt...
  Merging with facilities.txt...
  Merging with reported_events.txt...
Processing chunk 3...
  Merging with drop_withdrawals.txt...
  Merging with eligibilities.txt...
  Merging with facilities.txt...
  Merging with reported_events.txt...
Processing chunk 4...
  Merging with drop_withdrawals.txt...
  Merging with eligibilities.txt...
  Merging with facilities.txt...
  Merging with reported_events.txt...
Processing chunk 5...
  Merging with drop_withdrawals.txt...
  Merging with eligibilities.txt...
  Merging with facilities.txt...
  Merging with reported_events.txt...
Saving intermedia